Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на маллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.

Есть ли связь между жёсткостью воды и средней годовой смертностью? Посчитайте значение коэффициента корреляции Пирсона между этими признаками, округлите его до четырёх знаков после десятичной точки.

In [1]:
import numpy as np
import pandas as pd

In [2]:
df_water = pd.read_table('water.txt')

/Volumes/Storage/Applications/Programming/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [3]:
df_water.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [4]:
cities_water = df_water[['mortality', 'hardness']]

In [5]:
cities_water.head()

,mortality,hardness
0,1247,105
1,1668,17
2,1466,5
3,1800,14
4,1609,18


In [6]:
cities_water.corr()

,mortality,hardness
mortality,1.000000,-0.654849
hardness,-0.654849,1.000000


В предыдущей задаче посчитайте значение коэффициента корреляции Спирмена между средней годовой смертностью и жёсткостью воды. Округлите до четырёх знаков после десятичной точки. 

In [7]:
cities_water.corr(method='spearman')

,mortality,hardness
mortality,1.000000,-0.631665
hardness,-0.631665,1.000000


Сохраняется ли связь между признаками, если разбить выборку на северные и южные города? Посчитайте значения корреляции Пирсона между средней годовой смертностью и жёсткостью воды в каждой из двух подвыборок, введите наименьшее по модулю из двух значений, округлив его до четырёх знаков после десятичной точки.

In [8]:
north_cities_water = df_water[df_water['location'] == 'North'][['mortality', 'hardness']]
south_cities_water = df_water[df_water['location'] == 'South'][['mortality', 'hardness']]

In [9]:
north_cities_water.corr()

,mortality,hardness
mortality,1.000000,-0.368598
hardness,-0.368598,1.000000


In [10]:
south_cities_water.corr()

,mortality,hardness
mortality,1.000000,-0.602153
hardness,-0.602153,1.000000


Среди респондентов General Social Survey 2014 года хотя бы раз в месяц проводят вечер в баре 203 женщины и 239 мужчин; реже, чем раз в месяц, это делают 718 женщин и 515 мужчин.  

Посчитайте значение коэффициента корреляции Мэтьюса между полом и частотой похода в бары. Округлите значение до трёх знаков после десятичной точки.

In [11]:
a = 239
b = 515
c = 203
d = 718

In [12]:
mcc = (a * d - b * c) / np.sqrt((a + b) * (a + c) * (b + d) * (c + d))
print mcc

0.10900237458678963


В предыдущей задаче проверьте, значимо ли коэффициент корреляции Мэтьюса отличается от нуля. Посчитайте достигаемый уровень значимости; используйте функцию scipy.stats.chi2_contingency.

In [13]:
from scipy import stats

In [14]:
A = np.array([[a, b], [c, d]])

In [15]:
print 'p-value: ', stats.chi2_contingency(A)

p-value:  (19.40753078854304, 1.0558987006638725e-05, 1, array([[198.96597015, 555.03402985],
       [243.03402985, 677.96597015]]))


В предыдущей задаче давайте попробуем ответить на немного другой вопрос: отличаются ли доля мужчин и доля женщин, относительно часто проводящих вечера в баре? Постройте 95% доверительный интервал для разности долей, вычитая долю женщин из доли мужчин. Чему равна его нижняя граница? Округлите до четырёх знаков после десятичной точки.

In [16]:
men_selection = np.zeros((1, a + b))[0]
men_selection[0 : a] = 1
women_selection = np.zeros((1, c + d))[0]
women_selection[0 : c] = 1

In [17]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [18]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [19]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)

In [20]:
proportions_diff_confint_ind(men_selection, women_selection)

(0.053905233215813156, 0.13922183141523897)

Проверьте гипотезу о равенстве долей любителей часто проводить вечера в баре среди мужчин и женщин. Посчитайте достигаемый уровень значимости, используя двустороннюю альтернативу.

In [21]:
proportions_diff_z_test(proportions_diff_z_stat_ind(men_selection, women_selection))

8.153453089576601e-06

Посмотрим на данные General Social Survey 2014 года и проанализируем, как связаны ответы на вопросы "Счастливы ли вы?" и "Довольны ли вы вашим финансовым положением?"

|                  ||Не доволен|Более или менее|Доволен|
|-------------------|----------|---------------|-------|
|Не очень счастлив  |197       |111            |33     |
|Достаточно счастлив|382       |685            |331    |
|Очень счастлив     |110       |342            |333    |

Чему равно значение статистики хи-квадрат для этой таблицы сопряжённости?

In [22]:
happy_data_table = np.array([[197, 111, 33], [382, 685, 331], [110, 342, 333]])

In [23]:
chi2_contingency_returns = stats.chi2_contingency(happy_data_table)
chi_kv = chi2_contingency_returns[0]
print chi_kv

293.68311039689746


На данных из предыдущего вопроса посчитайте значение достигаемого уровня значимости.

In [24]:
p_value = chi2_contingency_returns[1]
print p_value

2.4964299580093467e-62


Чему в предыдущей задаче равно значение коэффициента V Крамера для рассматриваемых признаков?

In [25]:
n = np.sum(happy_data_table)

In [26]:
np.sqrt(chi_kv / (n * 2))

0.2412013934500338

In [27]:
stats.chi2_contingency(happy_data_table)

(293.68311039689746,
 2.4964299580093467e-62,
 4,
 array([[ 93.08597464, 153.74722662,  94.16679873],
        [381.6251981 , 630.318542  , 386.0562599 ],
        [214.28882726, 353.93423138, 216.77694136]]))